# HW1_Q7_Chami

## a) training model: learning class priors

In [43]:
import numpy as np

def prior(yTrain):
    classes = np.unique(yTrain)
    p = np.zeros(len(classes))

    for i, c in enumerate(classes):
        p[i] = np.sum(yTrain == c) / len(yTrain)

    return p

## b) training model — learning class-conditional feature probabilities

In [44]:
def likelihood(xTrain, yTrain):
    classes = np.unique(yTrain)
    c = len(classes)
    m = xTrain.shape[1]      # number of features

    M = np.zeros((m, c))
    V = np.zeros((m, c))

    for j, cls in enumerate(classes):
        Xc = xTrain[yTrain == cls]

        M[:, j] = np.mean(Xc, axis=0)
        V[:, j] = np.var(Xc, axis=0) 

    return M, V


## c) naive bayes classifier

In [45]:
def naiveBayesClassify(xTest, M, V, p):
    N = xTest.shape[0]      
    m, c = M.shape          

    pred = np.zeros(N)

    for n in range(N):
        post = np.zeros(c)

        for j in range(c):
            mu = M[:, j]
            sigma2 = V[:, j]

            pdf = (1 / np.sqrt(2 * np.pi * sigma2)) * np.exp(-((xTest[n] - mu)**2) / (2 * sigma2))
            post[j] = np.prod(pdf) * p[j]

        pred[n] = np.argmax(post) + 1   # +1 if classes start at 1

    return pred


## d) Evaluation

In [46]:
def evaluate(tPred, yTest):
    acc = np.mean(tPred == yTest)

    def precision_recall(k):
        TP = np.sum((tPred == k) & (yTest == k))
        FP = np.sum((tPred == k) & (yTest != k))
        FN = np.sum((tPred != k) & (yTest == k))

        precision = TP / (TP + FP) if TP + FP > 0 else 0
        recall = TP / (TP + FN) if TP + FN > 0 else 0
        return precision, recall

    p1, r1 = precision_recall(1)
    p5, r5 = precision_recall(5)

    with open("evaluation.txt", "w") as f:
        f.write(f"Accuracy: {acc:.3f}\n")
        f.write(f"Precision for class 1: {p1:.3f}\n")
        f.write(f"Recall for class 1: {r1:.3f}\n")
        f.write(f"Precision for class 5: {p5:.3f}\n")
        f.write(f"Recall for class 5: {r5:.3f}\n")


In [47]:
import scipy.io as sio

data = sio.loadmat("guassianBayes.mat")

xTrain = data["xTrain"]
yTrain = data["yTrain"].ravel()
xTest  = data["xTest"]
yTest  = data["yTest"].ravel()

p = prior(yTrain)
M, V = likelihood(xTrain, yTrain)
tPred = naiveBayesClassify(xTest, M, V, p)
evaluate(tPred, yTest)
